<a href="https://colab.research.google.com/github/ledyquesada/Ejercicios/blob/main/ISemana10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TÉCNICAS AVANZADAS DE OPTIMIZACIÓN EN MACHINE LEARNING
**SGD**

In [ ]:
#FUNCION SGD

import numpy as np
import random

def stochastic_gradient_descent(data, labels, model, learning_rate=0.01, epochs=100):
    for epoch in range(epochs):
        for i in range(len(data)):
            random_index = random.randint(0, len(data) - 1)
            x, y = data[random_index], labels[random_index]
            gradient = compute_gradient(x, y, model)
            model = update_parameters(model, gradient, learning_rate)
    return model


#COMPARACION

# Descenso de Gradiente Batch
def batch_gradient_descent(data, labels, model, learning_rate=0.01, epochs=100):
    for epoch in range(epochs):
        gradient = compute_gradient(data, labels, model)
        model = update_parameters(model, gradient, learning_rate)
    return model

# Descenso de Gradiente Mini-Batch
def mini_batch_gradient_descent(data, labels, model, batch_size=32, learning_rate=0.01, epochs=100):
    for epoch in range(epochs):
        for i in range(0, len(data), batch_size):
            mini_batch = data[i:i+batch_size]
            mini_labels = labels[i:i+batch_size]
            gradient = compute_gradient(mini_batch, mini_labels, model)
            model = update_parameters(model, gradient, learning_rate)
    return model


**OPTIMIZADORES POPULARES**

In [ ]:
#Momentum

from tensorflow.keras.optimizers import SGD

sgd_momentum = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=sgd_momentum, loss='categorical_crossentropy', metrics=['accuracy'])


#AdaGrad

from tensorflow.keras.optimizers import Adagrad

adagrad = Adagrad(learning_rate=0.01)
model.compile(optimizer=adagrad, loss='categorical_crossentropy', metrics=['accuracy'])

#RMSProp

from tensorflow.keras.optimizers import RMSprop

rmsprop = RMSprop(learning_rate=0.001, rho=0.9)
model.compile(optimizer=rmsprop, loss='categorical_crossentropy', metrics=['accuracy'])

#Adam

from tensorflow.keras.optimizers import Adam

adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])




**Ajuste de hiperparámetros**

In [1]:
#Tasa de aprendizaje

#Manual

learning_rates = [0.1, 0.01, 0.001]
for lr in learning_rates:
    optimizer = SGD(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#Busqueda en cuadricula

from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

def create_model(learning_rate=0.01):
    optimizer = SGD(learning_rate=learning_rate)
    model = Sequential()
    # Definir la arquitectura del modelo
    return model

model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)
learning_rates = [0.1, 0.01, 0.001]
param_grid = dict(learning_rate=learning_rates)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3)
grid_result = grid.fit(X_train, y_train)

#Momento

#Manual

momenta = [0.5, 0.9, 0.99]
for momentum in momenta:
    optimizer = SGD(learning_rate=0.01, momentum=momentum)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


#Aleatoria

from sklearn.model_selection import RandomizedSearchCV

def create_model(momentum=0.9):
    optimizer = SGD(learning_rate=0.01, momentum=momentum)
    model = Sequential()
    # Definir la arquitectura del modelo
    return model

model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)
momenta = [0.5, 0.9, 0.99]
param_dist = dict(momentum=momenta)
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, scoring='accuracy', cv=3)
random_result = random_search.fit(X_train, y_train)



**Transfer Learning**

In [4]:
#extraccion de caracterisitcas

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Utilizar el modelo base como extractor de características
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

#Ajuste fino

# Descongelar las últimas capas para ajuste fino
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Compilar el modelo para ajuste fino
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Ajuste fino en el nuevo conjunto de datos
model.fit(new_data, epochs=10)

# Metodos avanzados

from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense
from tensorflow.keras.models import Model

# Definir la entrada para las dos imágenes
input_a = Input(shape=(224, 224, 3))
input_b = Input(shape=(224, 224, 3))

# Construir la red siamesa
base_model = VGG16(weights='imagenet', include_top=False)
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
output = Dense(256)(x)

siamese_model = Model(inputs=base_model.input, outputs=output)

# Utilizar la misma instancia de modelo siamesa para ambas imágenes
output_a = siamese_model(input_a)
output_b = siamese_model(input_b)
